In [ ]:
#check if data exists
def check_if_data_exists(curr):
    query = ("""SELECT NAME FROM olasebik_acs_data WHERE NAME = %s""")
    curr.execute(query, (NAME,))
    
    return curr.fetchone() is not None #returns a single row from a table

#check to see if a data exists
def update_db(curr,NAME):
    tmp_df = pd.DataFrame(columns=['NAME', 'DPO2_0001E', 'DPO2_0002E', 'DPO2_0003E', 'DPO2_0015E', 'DPO2_0017E', 'DPO2_0098E','DPO2_0152E', 'state'])

    for i, row in CensusData.iterrows():
        if check_if_data_exists(curr):
            update_row(curr, row['NAME'], row['DPO2_0001E'], row['DPO2_0002E'], row['DPO2_0003E'], row['DPO2_0015E'], row['DPO2_0017E'], row['DPO2_0098E'],row['DPO2_0152E'], row['state'])
        else:  #The data does not exist so we will join to the database
            tmp_df = tmp_df.append(row)
    return tmp_df

#update row if data exists
def update_row(curr, NAME, DPO2_0001E, DPO2_0002E, DPO2_0003E, DPO2_0015E, DPO2_0017E, DPO2_0098E,DPO2_0152E, state):
    query = ("""UPDATE olasebik_acs_data
            SET NAME CHAR PRIMARY KEY,
                DPO2_0001E =%s,
                DPO2_0002E =%s,
                DPO2_0003E =%s,
                DPO2_0015E =%s,
                DPO2_0017E =%s,
                DPO2_0098E =%s,
                DPO2_0152E =%s,
                state =%s
            WHERE NAME = %S;""")
    vars_to_update = (NAME, DPO2_0001E, DPO2_0002E, DPO2_0003E, DPO2_0015E, DPO2_0017E, DPO2_0098E,DPO2_0152E, state)
    curr.execute(query, vars_to_update)
    
    
#write insert command

def insert_into_table(curr, NAME, DPO2_0001E, DPO2_0002E, DPO2_0003E, DPO2_0015E, DPO2_0017E, DPO2_0098E,DPO2_0152E, state):
    insert_into_data = ("""INSERT INTO olasebik_acs_data (NAME, DPO2_0001E, DPO2_0002E, DPO2_0003E, DPO2_0015E, DPO2_0017E, DPO2_0098E,DPO2_0152E, state)
    VALUES(%s, $s, $s, %s, %s, %s, %s);""")
    row_to_insert = (NAME, DPO2_0001E, DPO2_0002E, DPO2_0003E, DPO2_0015E, DPO2_0017E, DPO2_0098E,DPO2_0152E, state)
    curr.execute(insert_into_data, row_to_insert)
    
def append_from_df_to_db(curr,CensusData):
    for i, row in CensusData.iterrows():
        insert_into_table(curr, row['NAME'], row['DPO2_0001E'], row['DPO2_0002E'], row['DPO2_0003E'], row['DPO2_0015E'], row['DPO2_0017E'], row['DPO2_0098E'],row['DPO2_0152E'], row['state'])